<a href="https://colab.research.google.com/github/virf96/Chat-Bot/blob/master/DatosFaltantes_SustitucionMediaMediana_FeatureEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sustitución por la Media / Mediana  ==> Feature-engine


### Qué es Feature-engine?

Feature-engine es una librería de Python que hemos creado para este curso. 

- Feature-engine incluye todas las técnicas de ingeniería de variables descritas en este curso
- Feature-engine funciona como Scikit-learn, por lo tanto es fácil de aprender
- Feature-engine te permite implementar pasos de ingeniería de variables específicos para diferentes grupos de variables
- Feature-engine puede ser integrado con las pipelines de Scikit-learn pipeline permitiendo construir modelos fácilmente
** Feature-engine te permite diseñar y guardar un flujo de ingeniería de variables con procesos diseñados específicamente para diferentes grupos de variables.**

-------------------------------------------------------------------
Feature-engine puede ser instalado vía pip ==> pip install feature-engine

- Asegurate que haz instalado Feature-engine antes de correr este notebook

Para más detalle visita el [website de trainindata]( https://www.trainindata.com/feature-engine) 


## En este demo:

Vamos a usar ** Feature-engine para hacer la sustitución por la media o la mediana** usando los datos Ames House Price.

- Para bajar los datos, por favor referirse a la clase de **Datasets** en la  **Sección 1** del curso.

### Nota: 
* 'Imputer' deriva del verbo en inglés 'to impute' que quiere decir sustituir o reemplazar. Imputer es el objeto que completa la sustitución, de ahí el nombre dado a la clase.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# feature-engine
from feature_engine import missing_data_imputers as mdi

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# carguemos los datos con las variables seleccionadas

cols_to_use = [
    'BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt',
    'SalePrice'
]

data = pd.read_csv('/content/drive/My Drive/datasets/houseprice.csv', usecols=cols_to_use)
data.head()

,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,GarageYrBlt,SalePrice
0,65.0,196.0,Gd,NaN,2003.0,208500
1,80.0,0.0,Gd,TA,1976.0,181500
2,68.0,162.0,Gd,TA,2001.0,223500
3,60.0,0.0,TA,Gd,1998.0,140000
4,84.0,350.0,Gd,TA,2000.0,250000


In [ ]:
# porcentaje de valores nulos

data.isnull().mean()

LotFrontage    0.177397
MasVnrArea     0.005479
BsmtQual       0.025342
FireplaceQu    0.472603
GarageYrBlt    0.055479
SalePrice      0.000000
dtype: float64

Todas las variables predictivas tienen datos ausentes


In [ ]:
# separar datos en segmentos de entrenamiento y prueba

# primero, separemos el target (SalePrice) del resto de las variables
cols_to_use.remove('SalePrice')

X_train, X_test, y_train, y_test = train_test_split(data[cols_to_use],
                                                    data['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((1022, 5), (438, 5))

### Feature-engine captura las variables numéricas automáticamente

In [ ]:
# llamamos el imputer de Feature-engine
# especificamos la estrategia de sustitución, mediana en este caso

imputer = mdi.MeanMedianImputer(imputation_method='median')

In [ ]:
# ajustamos el imputer
imputer.fit(X_train)

MeanMedianImputer(imputation_method='median',
                  variables=['LotFrontage', 'MasVnrArea', 'GarageYrBlt'])

In [ ]:
# vemos que el imputer automáticamente encontró las variables numéricas para 
# sustituir con la media

imputer.variables

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [ ]:
# aquí podemos ver la mediana asignada a cada variable

imputer.imputer_dict_

{'LotFrontage': 69.0, 'MasVnrArea': 0.0, 'GarageYrBlt': 1979.0}

In [ ]:
# Feature-engine retorna un dataframe 

tmp = imputer.transform(X_train)
tmp.head()

,BsmtQual,FireplaceQu,LotFrontage,MasVnrArea,GarageYrBlt
64,Gd,NaN,69.0,573.0,1998.0
682,Gd,Gd,69.0,0.0,1996.0
960,TA,NaN,50.0,0.0,1979.0
1384,TA,NaN,60.0,0.0,1939.0
1100,TA,NaN,60.0,0.0,1930.0


In [ ]:
# revisemos que las variables numéricas no tengan 
# valores nulos NA 

tmp[imputer.variables].isnull().mean()

LotFrontage    0.0
MasVnrArea     0.0
GarageYrBlt    0.0
dtype: float64

## Feature-engine te permite especificar grupos de variables fácilmente

In [ ]:
# usemos la sustitución por la media 
# para 2 de la 3 variables numéricas

imputer = mdi.MeanMedianImputer(imputation_method='mean',
                                variables=['LotFrontage', 'MasVnrArea'])

imputer.fit(X_train)

MeanMedianImputer(imputation_method='mean',
                  variables=['LotFrontage', 'MasVnrArea'])

In [ ]:
# ahora el imputer solo imputa las variables que indicamos

imputer.variables

['LotFrontage', 'MasVnrArea']

In [ ]:
# y podemos ver el valor asignado a cada variable
imputer.imputer_dict_

{'LotFrontage': 69.66866746698679, 'MasVnrArea': 103.55358898721731}

In [ ]:
# corroboremos que el diccionario anterior contiene los valores promedio
# de las variables

X_train[imputer.variables].mean()

LotFrontage     69.668667
MasVnrArea     103.553589
dtype: float64

In [ ]:
# Feature-engine devuelve un dataframe

tmp = imputer.transform(X_train)

# miremos que los valores nulos efectivamente ya no existen
tmp[imputer.variables].isnull().mean()

LotFrontage    0.0
MasVnrArea     0.0
dtype: float64

## Feature-engine puede ser usado con los flujos de Scikit-learn (pipeline)

In [ ]:
pipe = Pipeline([
    ('median_imputer', mdi.MeanMedianImputer(imputation_method='median',
                                             variables = ['LotFrontage', 'GarageYrBlt'])),
     
    ('mean_imputer', mdi.MeanMedianImputer(imputation_method='mean',
                                          variables = ['MasVnrArea'])),
     ])

In [ ]:
pipe.fit(X_train)

Pipeline(memory=None,
         steps=[('median_imputer',
                 MeanMedianImputer(imputation_method='median',
                                   variables=['LotFrontage', 'GarageYrBlt'])),
                ('mean_imputer',
                 MeanMedianImputer(imputation_method='mean',
                                   variables=['MasVnrArea']))],
         verbose=False)

In [ ]:
pipe.named_steps['median_imputer'].imputer_dict_

{'LotFrontage': 69.0, 'GarageYrBlt': 1979.0}

In [ ]:
pipe.named_steps['mean_imputer'].imputer_dict_

{'MasVnrArea': 103.55358898721731}

In [ ]:
# transformemos los datos con la pipeline
tmp = pipe.transform(X_train)

# revisemos que ya no tenemos valores nulos
tmp.isnull().mean()

BsmtQual       0.023483
FireplaceQu    0.467710
LotFrontage    0.000000
MasVnrArea     0.000000
GarageYrBlt    0.000000
dtype: float64